In [68]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data


In [80]:
'''
Load  bag filesto make into respective dataframes
'''
defaultPath="/home/rhagoletis/catkin/src/World/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*traj.bag"])
print paths, "\n"
metadata=None
tic()
i=1
for path in paths:
    tic()
    print "starting analysis of file %s , %s / %s files" %(path.split('/')[-1],i,len(paths))
    df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
    bag=rosbag.Bag(path)
    
    try:
        for topic,msg,t in bag.read_messages(topics='/metadata'):
            a=msg
#         parameters=json.loads(a.data)
#         metadata={"meta":parameters}

        metadata=json.loads(a.data)

    except :
        print "no such file!, trying the pickle"
        metadata=depickler(paths[0].split('.bag')[0])
        parameters=metadata['parameters']
        
            
    obj=dict(df=df,metadata=metadata)
    
    picklepath=path+"_df.pickle"
    pickler(obj,picklepath)
#     df.to_pickle(picklepath)
    
    i+=1
    toc()
print "\nanalysis of %s files complete" %len(paths)
toc()

['/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly6_quad_rg_gain7.0_speed_3.5_trial_1_2016-07-06__10:30:14.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly6_wind_quad_00_gain7.0_speed_3.5_trial_1_2016-07-06__14:01:54.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly8_quad_00_gain10.42_speed_3.5_trial_1_2016-07-10__13:43:26.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly8_quad_gg_gain7.0_speed_3.5_trial_1_2016-07-09__18:07:48.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly8_quad_rg_gain7.0_speed_3.5_trial_1_2016-07-07__01:19:06.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly8_quad_rr_gain7.0_speed_3.5_trial_1_2016-07-09__23:46:22.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly8_wind_quad_00_gain7.0_speed_3.5_trial_1_2016-07-07__01:49:12.bag', '/media/rhagoletis/6db8b2b2-ebe2-4555-9b79

In [91]:
params=metadata

In [77]:
def loader():
    '''Load a dataframe from pickle and analyse lr cases'''

    defaultPath="/home/rhagoletis/catkin/src/World/bags/"
    path=easygui.fileopenbox(title="Choose pickle Dataframes"
                              ,default=defaultPath,
                              multiple=False,filetypes=["*.pickle"])
    print path

    dff=pd.read_pickle(path)
    df=dff['df']
    metadata=dff['metadata']
    return df,metadata,path

df,metadata,path=loader()
parameters=metadata['parameters']

/media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/Odour flies/fly8_wind_quad_00_gain7.0_speed_3.5_trial_1_2016-07-08__11:12:18.bag_df.pickle


KeyError: 'df'

In [83]:
'''
plot a single df
'''

from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import Slider
from bokeh.io import output_file, show, vform


slider = Slider(start=0, end=10, value=1, step=.1, title="Stuff")

# prepare some data
N = len(df.trajectory__position_x)
x = df.trajectory__position_x
y = df.trajectory__position_y
h=np.deg2rad(df.trajectory__orientation_x)

dec=10
x=x[::dec]
y=y[::dec]
h=h[::dec]
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

# output to static HTML file (with CDN resources)
output_file(path+".html", title="Trajectory Quad", mode="cdn")

TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,undo,redo,save"

# create a new plot with the tools above, and explicit ranges
p = figure(tools=TOOLS, x_range=(0,256), y_range=(0,256),webgl=True,plot_width=1000, plot_height=1000)
# p.image(image=[np.flipud(np.rot90(np.invert((parameters["odourField"])>0)*1))], 
#         x=0, y=0, dw=255, dh=255, palette="Greys9")

# add a circle renderer with vectorized colors and sizes
# p.circle(x,y,size=4  ,fill_alpha=0.5, line_color=None)

print vform(slider)
p.triangle(x,y,size=slider ,angle=h ,fill_alpha=0.8, line_color=None)
# p.wedge(x, y, radius=0.2, start_angle=0.4, end_angle=4.8,
#         color="firebrick", alpha=0.6, direction="clock")
# show the results
show(p)

WidgetBox(id='c115898e-0250-40dd-a843-efc856fc1ec2', ...)


/usr/local/lib/python2.7/dist-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: bokeh.io.vform() was deprecated in Bokeh 0.12.0 and will be removed, use bokeh.models.layouts.WidgetBox instead.
  warn(message)
/usr/local/lib/python2.7/dist-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: bokeh.models.layout.VBoxForm was deprecated in Bokeh 0.12.0 and will be removed, use bokeh.models.layouts.WidgetBox instead.
  warn(message)


ValueError: expected an element of either String, Dict(String, Either(String, Instance(Transform), Instance(ColorMapper), Float)) or Float, got Slider(id='8acf275c-2d81-41fd-b820-1c7b8aabcf59', ...)

In [75]:
parameters['odo']

{u'DCoffset': -0.288,
 u'DCoffsetIncrement': 0.012,
 u'area': 8,
 u'areaMode': True,
 u'bagFullTopics': u'/rhag_camera/image_raw/compressed /kinefly/image_output /servo_camera/image_raw/compressed /kinefly/flystate /trajectory  ',
 u'bagTrajTopics': u'/kinefly/flystate /trajectory ',
 u'bboxDist': 1.0,
 u'beepPath': u'/home/rhagoletis/catkin/src/World/models/sounds/beep.wav',
 u'boutNum': 2,
 u'camFOV': (120, 140),
 u'camHpr': (0, 10, 0),
 u'captureScale': 1,
 u'captureStart': 0,
 u'durList': [3, 3, 3, 3, 3, 3],
 u'durListGen': False,
 u'emergence': u'2000-01-01',
 u'factorDur': 0.65,
 u'fly': u'apple11',
 u'fps': 165,
 u'frameNum': 0,
 u'frameRecord': False,
 u'gain': 8,
 u'gainIncrement': 0.02,
 u'generateWorld': True,
 u'gpMode': True,
 u'greenTexPath': u'/home/transire/catkin/src/world/models/green.tga',
 u'headingListGen': False,
 u'headingRate': [0.25, 0.5, 1, 2, 4, 8],
 u'humanDisplay': False,
 u'imposeStimulus': False,
 u'interTrial': 3,
 u'intraTrial': 3,
 u'loadNullModels': F

In [50]:
pick=depickler(paths[0].split('.bag')[0])


In [65]:
df.to_csv('f.csv')

In [66]:
ls


20161031/       all.csv                  image.html
20161101/       als.csv                  lr analyser.ipynb
20161101_HIGH/  als.xls                  musca traj/
20161108/       Bag Plotter-Copy1.ipynb  musca trajectory analysis.ipynb
20161108_noon/  f                        processed_GC-PID_31-01-01high.csv
20161111/       f.csv                    temp.html
20161111_noon/  GC_PID_plotter.ipynb     test.py
20161112/       HeadingHist.ipynb        test.pyc
20161117/       histogram_color.html     Untitled.ipynb


In [6]:
'''
Load  bag filesto make into respective dataframes
1726819
'''
defaultPath="/home/rhagoletis/catkin/src/beginner/scripts/panda/world/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*traj.bag"])
print paths, "\n"
metadata=None

i=1
for path in paths:
    tic()
    print "starting analysis of file %s , %s / %s files" %(path.split('/')[-1],i,len(paths))
    df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
    bag=rosbag.Bag(path)
    
#     try:
#         for topic,msg,t in bag.read_messages(topics='/metadata'):
#             a=msg
# #         parameters=json.loads(a.data)
# #         metadata={"meta":parameters}
#         metadata=json.loads(a.data)

#     except :
#         print "no such file!"
            
#     obj=dict(df=df,metadata=metadata)
    
#     picklepath=path+"_df.pickle"
#     pickler(obj,picklepath)
# #     df.to_pickle(picklepath)
    
    i+=1
    toc()
print "\nanalysis of %s files complete" %len(paths)


None 



TypeError: 'NoneType' object is not iterable

In [ ]:
df

In [7]:
parameters=obj['metadata']['parameters']
print parameters

{u'interTrial': 3, u'stepMode': True, u'mouseMode': False, u'greenTexPath': u'/home/transire/catkin/src/world/models/green.tga', u'speed': 0.0, u'gainIncrement': 0.02, u'headingListGen': False, u'windDirection': 180, u'wbas': 0, u'wbad': 0, u'stepDur': -1, u'captureScale': 1, u'fps': 165, u'worldSize': 1025, u'startDur': 12, u'stopDur': 2, u'odour3Mask': u'', u'loadOdour': False, u'imposeStimulus': False, u'modelSizeSuffix': u'1025', u'pausePlot': True, u'frameNum': 0, u'trajectoryUpdateInterval': 30, u'windowHeight': 2560, u'startHeading': 2.1, u'phi': 0.0, u'object1': u'models/3D_Models/appletree/appletree_mp.egg', u'object2': u'models/3D_Models/appletree/appletree_mp_a.egg', u'loadNullModels': False, u'radius': 3.0, u'playbackIncrement': 1, u'packetDur': 0.006, u'captureStart': 0, u'loadTrajectory': False, u'camHpr': [0, 10, 0], u'nSteps': 5, u'race': u'apple', u'recordDur': 10, u'resetObject': False, u'speedIncrement': 0.05, u'odour4': u'models/odour/4.png', u'obj2Z': 0.0, u'odour1

In [ ]:

df[['trajectory__object1Pos_x', 'trajectory__object1Pos_y']]

In [ ]:
# plt.scatter(df.trajectory__position_x,df.trajectory__position_y)
# plt.show()
# df.trajectory__wbad.plot()
# df.trajectory__wbas.plot()
a=(pd.rolling_mean(df.trajectory__wbas,30))
a.plot()
df.trajectory__reset.plot()
plt.show()

In [ ]:
df

In [ ]:

def analyser(winTitle):
    
    '''
    takes window title input and returns average 
    compensation during stimulus period for each speed as a df
    '''
    #unknown rare bug, shabby fix
    df.trajectory__impose[0]=0
    
    impose=df.trajectory__impose.copy()
    impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
    compensation=df.trajectory__imposeResponse/impose
    df.trajectory__imposeResponseSmooth=pd.rolling_mean(df.trajectory__imposeResponse,window=165)
    compensationSmooth=df.trajectory__imposeResponseSmooth/impose

    cp = pd.DataFrame(index=compensation.index,columns=['a'])

    #flips are position where there is a rise or fall
    flips=np.sign(np.diff(df.trajectory__impose))

    #impose is impose df without first element
    impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

    #cw is clockWise, ccw is Counter clockwise
    #start and stop are positions where the impose starts or stops
    #start(stop) is zero everywhere except when there is the respective start(stop) event
    ccwStart=np.logical_and((flips==1) , (impose>0))
    ccwStop=np.logical_and((flips==-1) , (impose==0))
    cwStart=np.logical_and((flips==-1) , (impose<0))
    cwStop=np.logical_and((flips==1) , (impose==0))

#     plt.plot(np.array(impose),'r')
#     plt.plot(np.array(flips),'g')
#     plt.plot(np.array(ccwStart)*1,'b')
#     plt.plot(np.array(ccwStop)*1,'k')
#     plt.plot(np.array(cwStart)*1,'m')
#     plt.plot(np.array(cwStop)*1,'y')
#     print ccwStart.shape,ccwStop.shape
#     print cwStart.shape,cwStop.shape
#     plt.show()

    #(c)cwStart(stop)Index is just those positions where flips occured

    ccwStartIndex= ccwStart[ccwStart==1]
    cwStartIndex= cwStart[cwStart==1]
    ccwStopIndex= ccwStop[ccwStop==1]
    cwStopIndex= cwStop[cwStop==1]
#     print ccwStartIndex.shape,ccwStopIndex.shape
#     print cwStartIndex.shape,cwStopIndex.shape

#     print ccwStartIndex,ccwStopIndex
#     print cwStartIndex,cwStopIndex


    #compensationMean is a df who is zero when impose is false, 
    # and has mean value of compensation during the corresponding impose
    compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

    # print compensation
    def compMean(start,stop,compIn,compOut):
        consol=pd.DataFrame(columns=['fly','gain','impose','compensation'])


        print "calculating Mean"
        #if there are more starts than stop because of abrupt ending while recording
        #remove the last start event
        
        #remove the last events to remove noise
        start=start[:-1]
        stop=stop[:-1]

        try: 
            assert len(start)==len(stop)
        except AssertionError:
            print 'assertion error'
            start=start[:-1] 

        for i in range(len(stop)):
#             print "impose is",df.trajectory__impose[start.index[i]]
#             print "gain is",df.trajectory__gain[start.index[i]]


#             print "i mean is",compIn.ix[start.index[i]:stop.index[i]].mean()
            currCompensationMean=compIn.ix[start.index[i]:stop.index[i]].mean()
            compOut.ix[start.index[i]:stop.index[i]]=currCompensationMean
            curr=pd.DataFrame([{'fly':parameters['fly'],
                                                 "gain":df.trajectory__gain[start.index[i]],
                                                 "impose":df.trajectory__impose[start.index[i]],
                                                 'compensation':currCompensationMean}])
            consol=consol.append(curr,ignore_index=True)        
        cols=['fly','gain','impose','compensation']
        consol=consol[cols]
#         print "total measn were",i+1
        return consol

    cw=compMean(cwStartIndex,cwStopIndex,compensation,cp)
    ccw=compMean(ccwStartIndex,ccwStopIndex,compensation,cp)
    
    
    
    plt.plot(df.trajectory__imposeResponseSmooth,c=(0,0,1,0.2),label='imposeResponseSmooth')
#     plt.plot(compensationSmooth,c=(1,0,0,0.6),label='compensationSmooth')
    plt.plot(df.trajectory__impose,label='impose')
    plt.plot(cp,'g',label='compensationMean')
    plt.axhline(y=1)
    plt.title(winTitle)
    plt.legend()
    plt.show()
    
    return ccw, cw


In [ ]:

def analyser2(winTitle):
    
    '''
    takes window title input and returns average 
    compensation during stimulus period for each speed as a df
    '''
    #unknown rare bug, shabby fix
    df.trajectory__impose[0]=0
    
    impose=df.trajectory__impose.copy()
    impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
    imposeSign=np.sign(impose)
#     imposeSign[imposeSign==0]=1
    compensation=(df.trajectory__imposeResponse-df.trajectory__impose)*imposeSign/abs(impose)
    
    df.trajectory__imposeResponseSmooth=pd.rolling_mean(df.trajectory__imposeResponse,window=165)
    compensationSmooth=df.trajectory__imposeResponseSmooth/impose

    cp = pd.DataFrame(index=compensation.index,columns=['a'])

    #flips are position where there is a rise or fall
    flips=np.sign(np.diff(df.trajectory__impose))

    #impose is impose df without first element
    impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

    #cw is clockWise, ccw is Counter clockwise
    #start and stop are positions where the impose starts or stops
    #start(stop) is zero everywhere except when there is the respective start(stop) event
    ccwStart=np.logical_and((flips==1) , (impose>0))
    ccwStop=np.logical_and((flips==-1) , (impose==0))
    cwStart=np.logical_and((flips==-1) , (impose<0))
    cwStop=np.logical_and((flips==1) , (impose==0))

#     plt.plot(np.array(impose),'r')
#     plt.plot(np.array(flips),'g')
#     plt.plot(np.array(ccwStart)*1,'b')
#     plt.plot(np.array(ccwStop)*1,'k')
#     plt.plot(np.array(cwStart)*1,'m')
#     plt.plot(np.array(cwStop)*1,'y')
#     print ccwStart.shape,ccwStop.shape
#     print cwStart.shape,cwStop.shape
#     plt.show()

    #(c)cwStart(stop)Index is just those positions where flips occured

    ccwStartIndex= ccwStart[ccwStart==1]
    cwStartIndex= cwStart[cwStart==1]
    ccwStopIndex= ccwStop[ccwStop==1]
    cwStopIndex= cwStop[cwStop==1]
#     print ccwStartIndex.shape,ccwStopIndex.shape
#     print cwStartIndex.shape,cwStopIndex.shape

#     print ccwStartIndex,ccwStopIndex
#     print cwStartIndex,cwStopIndex


    #compensationMean is a df who is zero when impose is false, 
    # and has mean value of compensation during the corresponding impose
    compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

    # print compensation
    def compMean(start,stop,compIn,compOut):
        consol=pd.DataFrame(columns=['fly','gain','impose','compensation'])


#         print "calculating Mean"
        #if there are more starts than stop because of abrupt ending while recording
        #remove the last start event
        
        #remove the last events to remove noise
        start=start[:-1]
        stop=stop[:-1]

        try: 
            assert len(start)==len(stop)
        except AssertionError:
#             print 'assertion error'
            start=start[:-1] 

        for i in range(len(stop)):
#             print "impose is",df.trajectory__impose[start.index[i]]
#             print "gain is",df.trajectory__gain[start.index[i]]


#             print "i mean is",compIn.ix[start.index[i]:stop.index[i]].mean()
            currCompensationMean=compIn.ix[start.index[i]:stop.index[i]].mean()
            compOut.ix[start.index[i]:stop.index[i]]=currCompensationMean
            curr=pd.DataFrame([{'fly':parameters['fly'],
                                                 "gain":df.trajectory__gain[start.index[i]],
                                                 "impose":df.trajectory__impose[start.index[i]],
                                                 'compensation':currCompensationMean}])
            consol=consol.append(curr,ignore_index=True)        
        cols=['fly','gain','impose','compensation']
        consol=consol[cols]
#         print "total measn were",i+1
        return consol

    cw=compMean(cwStartIndex,cwStopIndex,compensation,cp)
    ccw=compMean(ccwStartIndex,ccwStopIndex,compensation,cp)
    
    
    
#     plt.plot(df.trajectory__imposeResponseSmooth,c=(0,0,1,0.2),label='imposeResponseSmooth')
    plt.plot(df.trajectory__imposeResponse,c=(0,0,1,0.2),label='imposeResponse')
#     plt.plot(compensationSmooth,c=(1,0,0,0.6),label='compensationSmooth')
    plt.plot(compensation,c=(1,0,0,0.6),label='compensation')

    plt.plot(df.trajectory__impose,label='impose')
    plt.plot(cp,'g',label='compensationMean',lw=4)
#     plt.axhline(y=1)
    plt.title(winTitle)
    plt.legend()
    plt.show()
    
    return ccw, cw


In [ ]:
a=analyser2(Wtitle)
# fullConsol=fullConsol.append(ccw,ignore_index=True)
# fullConsol=fullConsol.append(cw,ignore_index=True)

In [ ]:
'''Load a dataframe from pickle and analyse impose compensation'''
defaultPath="/home/rhagoletis/catkin/src/beginner/scripts/panda/world/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*df.pickle"])
fullConsol=pd.DataFrame()
# print paths, "\n"
for path in paths:
    
    tic()
    print "\n\ncurrentl;y analysisnh",path
    # picklepath=path+"_df.pickle"
    data=depickler(path)
    df=data["df"]
#     parameters=data["metadata"]['meta']
    parameters=data["metadata"]['parameters']

    toc()

    Wtitle=path.split('/')[-1]
    print Wtitle
    
#     print "parameter keys are", parameters.keys()
#     print "\nparameter keys length are", len(parameters.keys())

#     df.trajectory__imposeResponse.plot(title=path.split('/')[-1])
#     df.trajectory__impose.plot(title=path.split('/')[-1])
#     plt.show()
    
#     ccw,cw=analyser2(Wtitle)
#     fullConsol=fullConsol.append(ccw,ignore_index=True)
#     fullConsol=fullConsol.append(cw,ignore_index=True)
    
# print fullConsol

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pl

# Plot simple sinus function
fig_handle = plt.figure()
x = df.trajectory__position_x
y = df.trajectory__position_y

h=np.deg2rad(df.trajectory__orientation_x)

dec=1

x=x[::dec]
y=y[::dec]
h=h[::dec]
plt.scatter(x,y,marker='o',s=5,color=(1,0,0,0.1802))

# plt.scatter(initPl}ot.odd[:,0],initPlot.odd[:,1],color=initPlot.oddPlotColor,marker=initPlot.oddPlotMarker)
# plt.scatter(initPlot.even[:,0],initPlot.even[:,1],color=initPlot.evenPlotColor,marker=initPlot.evenPlotMarker),marker='|',color='g')
# plt.axis([0,255,0,255])

plt.show()


# Save figure handle to disk
# pl.dump(fig_handle,file('sinus.pickle','w'))



In [19]:
'''Load a dataframe from pickle'''
defaultPath="/home/rhagoletis/catkin/src/beginner/scripts/panda/world/bags/"
paths=easygui.fileopenbox(title="Bags to Dataframes"
                          ,default=defaultPath,
                          multiple=True,filetypes=["*df.pickle"])
fullConsol=pd.DataFrame()
# print paths, "\n"
for path in paths:
    
    tic()
    print "\n\ncurrentl;y analysisnh",path
    # picklepath=path+"_df.pickle"
    data=depickler(path)
    df=data["df"]
#     parameters=data["metadata"]['parameters']

    toc()





currentl;y analysisnh /media/rhagoletis/6db8b2b2-ebe2-4555-9b79-93b10ef9dec31/2016_08_08/traj/2016-08-08__22:34:07_fly8_quad_00_gain3.5_trial_1_traj.bag_df.pickle
Elapsed time: 1.148888 seconds.



In [20]:
plt.scatter(df.trajectory__position_x,df.trajectory__position_y)
plt.show()

In [16]:
maxdH=1
step=100
tot=1000


initX=10
initY=10
initH=90

prevH=initH
prevX=initX
prevY=initY

fullX=np.array([initX])
fullY=np.array([initY])

for i in range(tot):
    dH=maxdH*np.random.rand(1)
    newH=prevH+dH
    
    newX=prevX+step*np.cos(np.degrees(newH))
    newY=prevY+step*np.sin(np.degrees(newH))
    
    fullX=np.append(fullX,newX)
    fullY=np.append(fullY,newY)
    
    prevX=newX
    prevY=newY


# print fullX,fullY
plt.scatter(fullX,fullY)
plt.show()

In [ ]:
fullConsol=pd.DataFrame()
ccw,cw=analyser('a')
fullConsol=fullConsol.append(ccw,ignore_index=True)
fullConsol=fullConsol.append(cw,ignore_index=True)

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import Panel, Tabs
output_file("line.html")

x=df.trajectory__impose.index
y1=df.trajectory__impose
y2=df.trajectory__imposeResponse
y3=df.trajectory__imposeResponseSmooth
y4=compensationSmooth
y5=cp.fillna(0).abs()
# y5=cp

p1 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p2 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p3 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p4 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")
p5 = figure(plot_width=800, plot_height=800, y_range=(-3,3),x_axis_type="datetime")

p1.line(x,y1, line_width=2, line_color="navy")

p2.line(x,y2, line_width=0.75,line_color="orange",line_alpha=0.2)

p3.line(x,y1, line_width=2, line_color="navy")
p3.line(x,y2, line_width=0.75,line_color="orange",line_alpha=0.2)
p3.line(x,y3, line_width=1,line_color="olive",line_alpha=1)

p4.line(x,y1, line_width=2, line_color="navy")
p4.line(x,y3, line_width=0.75,line_color="orange",line_alpha=0.5)
p4.line(x,y4, line_width=1,line_color="olive",line_alpha=0.5)

p5.line(x,y5, line_width=2,line_color="darkgrey",line_alpha=1)
p5.line(x,y1, line_width=2, line_color="navy")
p5.line(x,y3, line_width=0.75,line_color="orange",line_alpha=0.5)
p5.line(x,y4, line_width=1,line_color="olive",line_alpha=0.5)


tab1 = Panel(child=p1, title="impose")
tab2 = Panel(child=p2, title="imposeResponse")
tab3 = Panel(child=p3, title="imposeResponseSmooth")
tab4 = Panel(child=p4, title="compensation")
tab5 = Panel(child=p5, title="compensationMean")




tabs = Tabs(tabs=[tab1,tab2,tab3,tab4 ,tab5])

show(tabs)


In [ ]:
# data['metadata']['parameters']['fly']
# print easygui.filesavebox()
parameters['fly']

In [ ]:
def consolidater():
    path=easygui.filesavebox()
    path+='_'+data['metadata']['parameters']['fly']+'_'+'consolidatedDF.pickle'

    print "save path is",path
    dataSet=dict(metadata=data['metadata'],fullConsol=fullConsol)
    pickler(dataSet,path)
consolidater()

In [ ]:
def unConsolidater():
    path=easygui.fileopenbox()

    print "file path is",path
    dataset=depickler(path)
unConsolidater()

In [ ]:
import seaborn as sns
# sns.set_style("whitegrid")

In [ ]:
ax = sns.stripplot(hue=fullConsol['impose'].round(1).abs(),   jitter=0.092,
                   y=fullConsol['compensation'],x=fullConsol['gain'],
                   split=True,palette="viridis",linewidth=1)
plt.axhline(y=-1)

plt.show()

In [ ]:
ax = sns.stripplot(x=fullConsol['impose'].round(1).abs(),   jitter=True,
                   y=fullConsol['compensation'],hue=fullConsol['gain'],
                   split=True,palette="viridis",linewidth=1)
plt.axhline(y=1)

plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.violinplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            hue=fullConsol["gain"], palette="viridis")
# sns.stripplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             hue=fullConsol["gain"], palette="viridis")
plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.violinplot(hue=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            x=fullConsol["gain"], palette="viridis")
# sns.stripplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             hue=fullConsol["gain"], palette="viridis")
plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.boxplot(hue=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            x=fullConsol["gain"], palette="viridis")
# sns.stripplot(hue=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             x=fullConsol["gain"], palette="viridis_r",jitter=True,split=True)
# plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
# sns.set(style="ticks")
sns.set_style("whitegrid")
# Draw a nested boxplot to show bills by day and sex
sns.boxplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
            hue=fullConsol["gain"], palette="viridis")
# sns.stripplot(x=fullConsol['impose'].round(1).abs() ,y=fullConsol["compensation"], 
#             hue=fullConsol["gain"], palette="viridis_r",jitter=True,split=True)
# plt.axhline(y=1)

sns.despine(offset=1, trim=True)
plt.show()

In [ ]:
pickler(fullConsol,'consolidatedDF')

In [ ]:
a=depickler('consolidatedDF')

In [ ]:
a['impose']=a['impose'].abs()
b=a.groupby(['fly','gain','impose'],as_index=False).mean()

ax = sns.stripplot(hue=b['impose'].round(1).abs(),   jitter=False,
                   y=b['compensation'],x=b['gain'],split=False,palette="viridis_r",size=8)
plt.show()

In [ ]:
plt.plot(b['gain'],b['compensation'])
plt.show()

In [ ]:
b['gain']
b['compensation']

In [ ]:


'''
takes window title input and returns average 
compensation during stimulus period for each speed as a df
'''
#unknown rare bug, shabby fix
df.trajectory__impose[0]=0

impose=df.trajectory__impose.copy()
impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
# compensation=df.trajectory__imposeResponse/impose
compensation=
compensationSmooth=df.trajectory__imposeResponseSmooth/impose

cp = pd.DataFrame(index=compensation.index,columns=['a'])

#flips are position where there is a rise or fall
flips=np.sign(np.diff(df.trajectory__impose))

#impose is impose df without first element
impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

#cw is clockWise, ccw is Counter clockwise
#start and stop are positions where the impose starts or stops
#start(stop) is zero everywhere except when there is the respective start(stop) event
ccwStart=np.logical_and((flips==1) , (impose>0))
ccwStop=np.logical_and((flips==-1) , (impose==0))
cwStart=np.logical_and((flips==-1) , (impose<0))
cwStop=np.logical_and((flips==1) , (impose==0))

#     plt.plot(np.array(impose),'r')
#     plt.plot(np.array(flips),'g')
#     plt.plot(np.array(ccwStart)*1,'b')
#     plt.plot(np.array(ccwStop)*1,'k')
#     plt.plot(np.array(cwStart)*1,'m')
#     plt.plot(np.array(cwStop)*1,'y')
#     print ccwStart.shape,ccwStop.shape
#     print cwStart.shape,cwStop.shape
#     plt.show()

#(c)cwStart(stop)Index is just those positions where flips occured

ccwStartIndex= ccwStart[ccwStart==1]
cwStartIndex= cwStart[cwStart==1]
ccwStopIndex= ccwStop[ccwStop==1]
cwStopIndex= cwStop[cwStop==1]
#     print ccwStartIndex.shape,ccwStopIndex.shape
#     print cwStartIndex.shape,cwStopIndex.shape

#     print ccwStartIndex,ccwStopIndex
#     print cwStartIndex,cwStopIndex


#compensationMean is a df who is zero when impose is false, 
# and has mean value of compensation during the corresponding impose
compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

# print compensation
def compMean(start,stop,compIn,compOut):
    consol=pd.DataFrame(columns=['fly','gain','impose','compensation'])


    print "calculating Mean"
    #if there are more starts than stop because of abrupt ending while recording
    #remove the last start event
    try: 
        assert len(start)==len(stop)
    except AssertionError:
        print 'assertion error'
        start=start[:-1] 

    for i in range(len(stop)):
#             print "impose is",df.trajectory__impose[start.index[i]]
#             print "gain is",df.trajectory__gain[start.index[i]]


#             print "i mean is",compIn.ix[start.index[i]:stop.index[i]].mean()
        currCompensationMean=compIn.ix[start.index[i]:stop.index[i]].mean()
        compOut.ix[start.index[i]:stop.index[i]]=currCompensationMean
        curr=pd.DataFrame([{'fly':parameters['fly'],
                                             "gain":df.trajectory__gain[start.index[i]],
                                             "impose":df.trajectory__impose[start.index[i]],
                                             'compensation':currCompensationMean}])
        consol=consol.append(curr,ignore_index=True)        
    cols=['fly','gain','impose','compensation']
    consol=consol[cols]
#         print "total measn were",i+1
    return consol

cw=compMean(cwStartIndex,cwStopIndex,compensation,cp)
ccw=compMean(ccwStartIndex,ccwStopIndex,compensation,cp)



#     plt.plot(df.trajectory__imposeResponseSmooth)
#     plt.plot(compensation,c=(1,0,0,0.2))
#     plt.plot(df.trajectory__impose)
# #     plt.plot(cp,'g')
#     plt.axhline(y=1)
#     plt.title(winTitle)
#     plt.show()



In [ ]:
impose=df.trajectory__impose.copy()
impose[df.trajectory__impose==0]=1 #replace impose zero with one fordiv by zero
compensation=df.trajectory__imposeResponseSmooth/impose
# plt.plot(df.trajectory__imposeResponseSmooth)
plt.plot(compensation,c=(1,0,0,0.2))
plt.plot(df.trajectory__impose)
plt.plot(cp,'g')
plt.axhline(y=1)
plt.show()

In [ ]:
parameters

In [ ]:
#flips are position where there is a rise or fall
flips=np.sign(np.diff(df.trajectory__impose))

#impose is impose df without first element
impose=df.trajectory__impose.drop(df.trajectory__impose.index[0])

#cw is clockWise, ccw is Counter clockwise
#start and stop are positions where the impose starts or stops
#start(stop) is zero everywhere except when there is the respective start(stop) event
ccwStart=np.logical_and((flips==1) , (impose>0))
ccwStop=np.logical_and((flips==-1) , (impose==0))
cwStart=np.logical_and((flips==-1) , (impose<0))
cwStop=np.logical_and((flips==1) , (impose==0))

# plt.plot(np.array(impose),'r')
# plt.plot(np.array(flips),'g')
# plt.plot(np.array(ccwStart)*1,'b')
# plt.plot(np.array(ccwStop)*1,'k')
# plt.plot(np.array(cwStart)*1,'m')
# plt.plot(np.array(cwStop)*1,'y')
# print ccwStart.shape,ccwStop.shape
# print cwStart.shape,cwStop.shape
plt.show()

#(c)cwStart(stop)Index is just those positions where flips occured

ccwStartIndex= ccwStart[ccwStart==1]
cwStartIndex= cwStart[cwStart==1]
ccwStopIndex= ccwStop[ccwStop==1]
cwStopIndex= cwStop[cwStop==1]
print ccwStartIndex.shape
print ccwStopIndex.shape

print cwStartIndex.shape
print cwStopIndex.shape


cp = pd.DataFrame(index=compensation.index,columns=['a'])

#compensationMean is a df who is zero when impose is false, 
# and has mean value of compensation during the corresponding impose
compensationMean = pd.DataFrame(index=compensation.index,columns=['mean']) #empty df with same index

# print compensation
def compMean(start,stop,compIn,compOut):
    print "calculating Mean"
    #if there are more starts than stop because of abrupt ending while recording
    #remove the last start event
    try: 
        assert len(start)==len(stop)
    except AssertionError:
        print 'assertion error'
        start=start[:-1] 

    for i in range(len(stop)):

        print "i mean is",compIn.ix[start.index[i]:
                                          stop.index[i]].mean()
        compOut.ix[start.index[i]:stop.index[i]]=compIn.ix[start.index[i]:stop.index[i]].mean()

    print "total measn were",i+1
compMean(cwStartIndex,cwStopIndex,compensation,cp)
compMean(ccwStartIndex,ccwStopIndex,compensation,cp)

In [ ]:
df.trajectory__imposeResponse.plot()
df.trajectory__impose.plot()

plt.show()

In [ ]:
plt.plot(pd.rolling_mean((df.trajectory__imposeResponse),window=10))
plt.plot(pd.rolling_mean((df.trajectory__imposeResponse),window=100),'r')
plt.show()
         

In [ ]:
# ccwStart=pd.DataFrame(0, index=np.arange(len(df)), columns=["feature_list"])
flips=np.sign(df.trajectory__impose.diff)
ccwStart[flips>0]=1
plt.plot(flips)
plt.show()

In [ ]:
flips=np.sign(np.diff(df.trajectory__impose))
impose=df.trajectory__impose.drop(df.trajectory__impose.index[1])
# a=np.roll(a,-1)
ccwStart=np.logical_and((flips==1) , (impose>0))
ccwStop=np.logical_and((flips==-1) , (impose==0))
cwStart=np.logical_and((flips==-1) , (impose<0))
cwStop=np.logical_and((flips==1) , (impose==0))

plt.plot(np.array(impose),'r')
plt.plot(np.array(flips),'g')
plt.plot(np.array(ccwStart)*1,'b')
plt.plot(np.array(ccwStop)*1,'k')
plt.plot(np.array(cwStart)*1,'m')
plt.plot(np.array(cwStop)*1,'y')
print ccwStart.shape,ccwStop.shape
print cwStart.shape,cwStop.shape

ccwStart= ccwStart[ccwStart==1]
cwStart= cwStart[cwStart==1]
ccwStop= ccwStop[ccwStop==1]
cwStop= cwStop[cwStop==1]
print ccwStart,ccwStop

plt.show()

In [ ]:
cp = pd.DataFrame(index=compensation.index,columns=['a'])
# print compensation
for i in range(len(ccwStart)):
    try: 
        assert len(ccwStart)==len(ccwStop)
    except AssertionError:
        ccwStop=ccwStop[:-1]
    
    print "i mean is",compensation.ix[cwStart.index[i]:
                                      cwStop.index[i]].mean()
    cp.ix[cwStart.index[i]:cwStop.index[i]]=compensation.ix[cwStart.index[i]:cwStop.index[i]].mean()
    

In [ ]:
a={'a':1,'b':2,'c':3,'d':5}
b={'a':2,'b':2,'c':3,'d':5}

# d.append(df, ignore_index=True)
df=pd.DataFrame([a])
df=df.append(pd.DataFrame([b]))#,ignore_index=True)
print df
# print d

In [ ]:
a

In [ ]:
print (df.trajectory__impose.shape)
a.shape

In [ ]:
(.3195+.34179)/2

In [ ]:
plt.plot(flips)
plt.plot(np.array(b),'r')

plt.show()

In [ ]:
df.trajectory__wbad.plot.hist(bins=60)

In [ ]:
''' 
Load multiple dataframe pickle file into memory and append it
'''
paths=easygui.fileopenbox(multiple=True,filetypes=['*.pickle'])
i=0
a=range(len(paths))
for path in paths:
    a[i]=pd.read_pickle(path)

    i+=1
    
df=pd.concat(a)

In [ ]:
#load params pickle
path=easygui.fileopenbox()
import cPickle as pl
import json
a=pl.load(open( path, "rb" ))
params=a[1]

In [ ]:
parameters


In [ ]:
df.trajectory__trial.max()

In [ ]:
''' 
Load multiple bag files to be converted to dataframe
'''

paths=easygui.fileopenbox(multiple=True,filetypes=['*.bag'])
i=0
for path in paths:

    df=rosbag_pandas.bag_to_dataframe(path,include=['/trajectory'])
    df.columns
    bag=rosbag.Bag(path)
    for topic,msg,t in bag.read_messages(topics='/metadata'):
        a=msg
    parameters=json.loads(a.data)
    picklepath=path+"_df.pickle"
    df.to_pickle(picklepath)
    i+=1
    
print "done"

In [ ]:
'''
plot a single df
'''

from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import Slider
from bokeh.io import output_file, show, vform


slider = Slider(start=0, end=10, value=1, step=.1, title="Stuff")

# prepare some data
N = len(df.trajectory__position_x)
x = df.trajectory__position_x
y = df.trajectory__position_y
h=np.deg2rad(df.trajectory__orientation_x)

dec=1
x=x[::dec]
y=y[::dec]
h=h[::dec]
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

# output to static HTML file (with CDN resources)
output_file(path+".html", title="Trajectory Quad", mode="cdn")

TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,undo,redo,save"

# create a new plot with the tools above, and explicit ranges
p = figure(tools=TOOLS, x_range=(0,256), y_range=(0,256),webgl=True,plot_width=1000, plot_height=1000)
p.image(image=[np.flipud(np.rot90(np.invert((parameters["odourField"])>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

# add a circle renderer with vectorized colors and sizes
# p.circle(x,y,size=4  ,fill_alpha=0.5, line_color=None)

print vform(slider)
p.triangle(x,y,size=slider ,angle=h ,fill_alpha=0.8, line_color=None)
# p.wedge(x, y, radius=0.2, start_angle=0.4, end_angle=4.8,
#         color="firebrick", alpha=0.6, direction="clock")
# show the results
show(p)

In [ ]:

from bokeh.models.widgets import Slider
from bokeh.io import output_file, show, vform

output_file("slider.html")

slider = Slider(start=0, end=10, value=1, step=.1, title="Stuff")

show(vform(slider))


import numpy as np

from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource

x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.get('data');
    var A = amp.get('value')
    var k = freq.get('value')
    var phi = phase.get('value')
    var B = offset.get('value')
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = B + A*Math.sin(k*x[i]+phi);
    }
    source.trigger('change');
""")

amp_slider = Slider(start=0.1, end=10, value=1, step=.1,
                    title="Amplitude", callback=callback)
callback.args["amp"] = amp_slider

freq_slider = Slider(start=0.1, end=10, value=1, step=.1,
                     title="Frequency", callback=callback)
callback.args["freq"] = freq_slider

phase_slider = Slider(start=0, end=6.4, value=0, step=.1,
                      title="Phase", callback=callback)
callback.args["phase"] = phase_slider

offset_slider = Slider(start=-5, end=5, value=0, step=.1,
                       title="Offset", callback=callback)
callback.args["offset"] = offset_slider

layout = row(
    plot,
    widgetbox(amp_slider, freq_slider, phase_slider, offset_slider),
)

output_file("slider.html", title="slider.py example")

show(layout)

In [ ]:
'''plotdoor field'''
p = figure(x_range=(0, 255), y_range=(0,255))

parameters["odourField"]=np.array(parameters["odourField"]['__ndarray__'])

p.image(image=[np.flipud(np.rot90(rescale(parameters['odourField'], 10)))], x=0, y=0, dw=255, dh=255, palette="Greys3")


# must give a vector of image data for image parameter

output_file("image.html", title="image.py example")

show(p)  # open a browser

In [ ]:
# parameters["odourField"]=parameters["odourField"]['__ndarray__']
parameters['odourField']=np.array((parameters["odourField"]))
from skimage.transform import rescale
rescale(parameters['odourField'], 10).shape


In [ ]:
#load df pickle
path=easygui.fileopenbox()
df=pd.read_pickle(path)
path=path.rstrip("_df.pickle")

In [ ]:
files=easygui.fileopenbox("select the files to be dataframed and plotted",multiple=True)

In [118]:
import seaborn as sns

In [117]:
'''
plot a single df
'''
params=obj['metadata']
from bokeh.plotting import figure, output_file, show, gridplot

# prepare some data
N = len(df.trajectory__position_x)
x = df.trajectory__position_x
y = df.trajectory__position_y
h=np.deg2rad(df.trajectory__orientation_x)
of=np.array([params["odourField"]["__ndarray__"]]).reshape(257,257)
dec=3
x=x[::dec]
y=y[::dec]
h=h[::dec]
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

# output to static HTML file (with CDN resources)
output_file(path+".html", title="Trajectory Quad", mode="cdn")

TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select,undo,redo,save"

# create a new plot with the tools above, and explicit ranges
w=1400
h=1400
s=5
bl = figure(tools=TOOLS, x_range=(64-32,64+32), y_range=(32-32,32+32),webgl=True,plot_width=w, plot_height=h)
tl = figure(tools=TOOLS, x_range=(64-32,64+32), y_range=(161-32,161+32),webgl=True,plot_width=w, plot_height=h)
br = figure(tools=TOOLS, x_range=(193-32,193+32), y_range=(32-32,32+32),webgl=True,plot_width=w, plot_height=h)
tr = figure(tools=TOOLS, x_range=(193-32,193+32), y_range=(161-32,161+32),webgl=True,plot_width=w, plot_height=h)

bl.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)
tl.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)
br.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)
tr.triangle(x,y,size=s ,angle=h ,fill_alpha=1, line_color=None)


bl.image(image=[np.flipud(np.rot90(np.invert((of)>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")


tl.image(image=[np.flipud(np.rot90(np.invert((of)>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

br.image(image=[np.flipud(np.rot90(np.invert((of)>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

tr.image(image=[np.flipud(np.rot90(np.invert((of)>0)*1))], 
        x=0, y=0, dw=255, dh=255, palette="Greys9")

p = gridplot([[tl, tr], [bl, br]])

# add a circle renderer with vectorized colors and sizes
# p.circle(x,y,size=4  ,fill_alpha=0.5, line_color=None)
# p.triangle(x,y,size=4 ,angle=h ,fill_alpha=1, line_color=None)
# p.wedge(x, y, radius=0.2, start_angle=0.4, end_angle=4.8,
#         color="firebrick", alpha=0.6, direction="clock")
# show the results
show(p)

In [148]:
fig, ax = plt.subplots(figsize=(12,12))
plt.imshow(np.flipud(np.rot90(of,3)),alpha=0.2,origin='lower')
ax.scatter(x,y,lw=0,s=1,c=[0,0,0.8,0.5],marker='o')
ax.set_aspect('equal', 'datalim')
plt.show()

In [122]:
import matplotlib as mpl
# df=metadata=None
def quadPlotter(df,metadata,pth='plot.pdf',col=[0,0,0,1],off=20,
                xmin=506,xmax=520,ymin=508,ymax=522,rm=10,s=4,xt='X (m)',yt='Y (m)'):
    
    parameters=metadata['parameters']
        # row and column sharing
    fig, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(14,14))
    ax={4:ax4,1:ax1,2:ax2,3:ax3}
    maxS=df.trajectory__speed.max()
    # ax1.set_title('lr anal')
    for i in range(int(df.trajectory__case.max()+1)):
        df2=df[(df.trajectory__case==i) & (df.trajectory__runNum<20) &(df.trajectory__isFlying==True)]
        c1=[1,0,0,0.5]
        c2=[0,1,0,0.5]

#         circles(df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0],ax=ax[i+1],color=c1,lw=0,s=parameters['obj1Scale']/2)
#         circles(df2.trajectory__o2Pos_x[0],df2.trajectory__o2Pos_y[0],ax=ax[i+1],color=c2,lw=0,s=parameters['obj1Scale']/2)
        sc=0.7
        r1=sc*parameters['obj1Scale']/2
        r2=sc*parameters['obj2Scale']/2
        
        ls=parameters['loadingString']
        non=[0,0,0,0]
        red=[1,0,0,0.3]
        green=[0,1,0,0.3]
        wf=3 #wifth fat
        w0=0 #width thin
        
        if ls=='00':
            l=non
            r=non
            w1=wf
            w2=wf
        elif ls=='01':
            l=non
            r=red
            w1=wf
            w2=w0
        elif ls=='10':
            l=green
            r=non
            w1=w0
            w2=wf
        elif ls=='11':
            l=green
            r=red
            w1=w0
            w2=w0

        if df2.trajectory__case[0] == 0:
            c1=l
            c2=l
            l1=w1
            l2=w1
        elif df2.trajectory__case[0]== 1:
            c1=r
            c2=l
            l1=w2
            l2=w1
        elif df2.trajectory__case[0]== 2:
            c1=l
            c2=r
            l1=w1
            l2=w2
        elif df2.trajectory__case[0] == 3:
            c1=r
            c2=r
            l1=w2
            l2=w2
        print 'c1 c2 is',c1,c2
        
#         ax[i+1].add_artist(Circle(xy=(parameters['posL'][0],parameters['posL'][1]), color=c1, radius=r1)) 

#         ax[i+1].add_artist(Circle(xy=(parameters['posR'][0],parameters['posR'][1]), color=c2, radius=r2)) 
            
        
        if df.trajectory__speedControl[0]:
            col=df2.trajectory__speed.values/maxS
        else:
            col=col
            
        im=ax[i+1].scatter(df2.trajectory__pPos_x[::rm],df2.trajectory__pPos_y[::rm],lw=0,s=s
                        ,c=col,cmap='viridis',rasterized=True)
        
        ax[i+1].scatter(parameters['playerInitPos'][0],parameters['playerInitPos'][1],c=np.array([62,218,0,200])/255.,
                        marker='^',lw=0,s=500)

        ax[i+1].scatter(parameters['posL'][0],parameters['posL'][1],c=c1,lw=l1,s=sc*parameters['obj1Scale'])
        ax[i+1].scatter(parameters['posR'][0],parameters['posR'][1],c=c2,lw=l2,s=sc*parameters[ 'obj2Scale'])

        ax[i+1].set_xlim([xmin,xmax])
        ax[i+1].set_ylim([ymin,ymax])

        ax[i+1].set_aspect('equal', 'datalim')
#         ax[i+1].set_title(i)
        
        ax[i+1].set_xlabel(xt)
        ax[i+1].set_ylabel(yt)

        print i
    
#     # Colorbar
#     ax3.cax.colorbar(ax3)
#     ax3.cax.toggle_label(True)
#     fig.colorbar(im, ax=ax3)
    
#     cax,kw = mpl.colorbar.make_axes([ax1,ax2,ax3,ax4])
#     plt.colorbar(im, cax=cax, **kw)
        plt.tight_layout()
#         fig.text(0.5, 0.04, xt, ha='center')
#         fig.text(0.04, 0.5, yt, va='center', rotation='vertical')

#     sns.despine()
#     sns.set_style("ticks")
#         sns.set_style("white")
#         sns.set_style("whitegrid")


#         sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
#     fig.subplots_adjust(right=0.8)
    
    if df.trajectory__speedControl[0]:

        cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
        fig.colorbar(im, cax=cbar_ax)
    
    plt.savefig(pth, dpi=400)

    plt.show()
    return fig


In [99]:
(params["odourField"].keys())

[u'dtype', u'shape', u'__ndarray__']

In [87]:
params.keys()

[u'interTrial',
 u'stepMode',
 u'mouseMode',
 u'greenTexPath',
 u'speed',
 u'gainIncrement',
 u'headingListGen',
 u'windDirection',
 u'wbas',
 u'wbad',
 u'stepDur',
 u'captureScale',
 u'plotOdourQuad',
 u'worldSize',
 u'startDur',
 u'stopDur',
 u'odour3Mask',
 u'loadOdour',
 u'imposeStimulus',
 u'modelSizeSuffix',
 u'pausePlot',
 u'frameNum',
 u'nReps',
 u'minFlightSpeed',
 u'startHeading',
 u'modelHeightMap',
 u'object1',
 u'object2',
 u'loadNullModels',
 u'radius',
 u'playbackIncrement',
 u'packetDur',
 u'captureStart',
 u'loadTrajectory',
 u'camHpr',
 u'nSteps',
 u'race',
 u'recordDur',
 u'resetObject',
 u'speedIncrement',
 u'odour4',
 u'obj2Z',
 u'odour1',
 u'lockFps',
 u'odour3',
 u'odour2',
 u'boutNum',
 u'windowHeight',
 u'maxDistance',
 u'posL',
 u'useOdourMask',
 u'maxWbas',
 u'skyMapNull',
 u'replayWorld',
 u'bboxDist',
 u'posR',
 u'gpMode',
 u'origin',
 u'durList',
 u'frameRecord',
 u'odour1Mask',
 u'beepPath',
 u'trajectoryUpdateInterval',
 u'skyMap',
 u'durListGen',
 u'are

In [ ]:
a=range(0,15)
a=a[::5]
a

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

N = 1000
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)

p = figure(x_range=(0, 10), y_range=(0, 10))

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, color_mapper=1)

output_file("image.html", title="image.py example")

show(p)  # open a browser

In [ ]:
from hcpWorldGen import WorldGen
df.plot.scatter('trajectory__position_x',y='trajectory__position_y',s=0.01)
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pl

# Plot simple sinus function
fig_handle = plt.figure()
x = df.trajectory__position_x
y = df.trajectory__position_y

h=np.deg2rad(df.trajectory__orientation_x)

dec=1

x=x[::dec]
y=y[::dec]
h=h[::dec]
plt.scatter(x,y,marker='o',s=5,color=(1,0,0,0.1802))

# plt.scatter(initPlot.odd[:,0],initPlot.odd[:,1],color=initPlot.oddPlotColor,marker=initPlot.oddPlotMarker)
# plt.scatter(initPlot.even[:,0],initPlot.even[:,1],color=initPlot.evenPlotColor,marker=initPlot.evenPlotMarker),marker='|',color='g')
# plt.axis([0,255,0,255])

plt.show()


# Save figure handle to disk
# pl.dump(fig_handle,file('sinus.pickle','w'))



In [ ]:
print type(fig_handle)

In [ ]:
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11")

In [ ]:
# parameters["odourField"]=np.array(parameters["odourField"]['__ndarray__'])
x = df.trajectory__position_x
y = df.trajectory__position_y
c=df.trajectory__valve
h=np.deg2rad(df.trajectory__orientation_x)

dec=3

x=x[::dec]
y=y[::dec]
h=h[::dec]
c=c[::dec]

plt.imshow(np.flipud(np.rot90(parameters["odourField"])),cmap='Greys')
plt.scatter(x,y,marker='o',s=5,c=c,cmap='seismic',lw=0)

plt.show()

In [ ]:
df.trajectory__valve.hist()
plt.show()

In [ ]:
'''plot hex bin'''

df.plot.hexbin(x='trajectory__position_x',y='trajectory__position_y',gridsize=100,cmap=plt.cm.viridis,)
plt.show()


In [ ]:
WorldGen.initPositions

In [ ]:
df.plot.hexbin(x='trajectory__position_x',y='trajectory__position_y',gridsize=100)

In [ ]:
plt.show()

In [ ]:
df.columns

In [ ]:
plt.imshow(np.flipud(np.rot90(parameters["odourField"])))
plt.show()

In [ ]:
params["odourField"].shape

In [ ]:
params

In [ ]:
a[1]

In [ ]:
2+3

In [ ]:
print paths

In [ ]:
# modules
#------------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as py
from matplotlib import animation

py.close('all') # close all previous plots

# create a random line to plot
#------------------------------------------------------------------------------

x = df.trajectory__position_x
y = df.trajectory__position_y
t = np.linspace(0,1,x.shape[0]) # your "time" variable
# py.figure(1)
# py.scatter(0, 0, s=1)
# py.axis([0, 255, 0, 255])
# py.show()

# animation of a scatter plot using x, y from above
#------------------------------------------------------------------------------

fig = py.figure(1)
ax = py.axes(xlim=(0, 255), ylim=(0,255))
scat = ax.scatter([], [], s=20,marker=(3,0,0))

def init():
    scat.set_offsets([])
    return scat,

def animate(i):
    data = np.hstack((x[:i,np.newaxis], y[:i, np.newaxis]))
    scat.set_offsets(data)
    scat.se
#     scat.set_marker('o')
    return scat,

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(x)+1, 
                               interval=1000/165., blit=True, repeat=False)
plt.show()

In [ ]:
len(df.trajectory__position_x)

In [ ]:
i=360


In [ ]:
import matplotlib


In [ ]:
print matplotlib.__version__